# GitHubに保存した教材バックアップをリストアします
---

GitHubリポジトリに保存してある教材バックアップをリストアします。

# リストアの準備

このNotebookを実行すると、GitHubリポジトリの教材バックアップをホームディレクトリ直下に展開します。

**リストアするには下記のフォルダ/ファイルは、全てバックアップの内容で上書きされます。**
* `textbook`
* `info`
* `draft`
* `test`
* 任意でバックアップに追加したフォルダ/ファイル

# GitHubリポジトリへの接続情報を設定します

バックアップ先のGitHubリポジトリへの接続情報を設定します。

1. GitHubに接続するSSHキーを新たに登録する場合は、 [GitHubリポジトリの準備/2. SSHキーの登録](03_PrepareGitHub.ipynb#create_ssh_key)を使って作成して下さい。
2. [GitHubリポジトリの準備/3. GitHubリポジトリ接続情報の設定](03_PrepareGitHub.ipynb#connect_github_repo)を使って、バックアップ用のGitHubリポジトリの接続情報を設定して下さい。

# 次のセルを実行して、教材バックアップを展開して下さい

GitHubから教材バックアップを取得して、ホームディレクトリに展開します。

In [ ]:
%%bash

cd
comm -13 <(sort ~/.ssh/known_hosts) <(ssh-keyscan github.com 2>/dev/null | sort) >>~/.ssh/known_hosts

IS_INSIDE_REPO=0
S="$(git rev-parse --is-inside-work-tree 2>/dev/null)"
[ $? -eq 0 -a "${S}" = "true" ] && IS_INSIDE_REPO=1

if [ ${IS_INSIDE_REPO} -ne 1 ]; then
  echo "ERROR: git repository not initialized." 1>&2
  exit 1
fi

git fetch --force origin
git reset --hard origin/master

git ls-files -c | while read -r path; do
  if [ -f "${path}" -a "${path: -6}" = ".ipynb" ]; then
    jupyter-trust "${path}"
  fi
done